# TRANSFORMACIÓN DE DATOS

## IMPORTAR PAQUETES

In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder

#Automcompletar rápido
%config IPCompleter.greedy=True

## IMPORTAR LOS DATOS

1.- Sustituir la ruta del proyecto.

In [49]:
ruta_proyecto = 'C:/Users/isaac/Google Drive/DS4B/CursoMachineLearningPython/03_MACHINE_LEARNING/08_CASOS/02_RETAIL'

2.- Nombrar los ficheros de datos.

In [50]:
nombre_cat = 'cat_resultado_eda.pickle'
nombre_num = 'num_resultado_eda.pickle'

3.- Cargar los datos.

In [51]:
cat = pd.read_pickle(ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_cat)
num = pd.read_pickle(ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_num)

## NUEVAS VARIABLES

En este caso ya nos vienen creadas:

* componentes de la fecha
* variables de calendario

Vamos a crear:

* las que identificamos en EDA
* lags
* ventanas móviles

Para esta fase necesitamos volver a unir en un solo dataframe.

In [52]:
df = pd.concat([cat,num], axis = 1)
df

store_id      item_id       d  year month wday  weekday  \
date                                                                 
2013-01-01     CA_3  FOODS_3_090   d_704  2013     1    4  Tuesday   
2013-01-01     CA_3  FOODS_3_120   d_704  2013     1    4  Tuesday   
2013-01-01     CA_3  FOODS_3_202   d_704  2013     1    4  Tuesday   
2013-01-01     CA_3  FOODS_3_252   d_704  2013     1    4  Tuesday   
2013-01-01     CA_3  FOODS_3_288   d_704  2013     1    4  Tuesday   
...             ...          ...     ...   ...   ...  ...      ...   
2015-11-30     CA_4  FOODS_3_329  d_1767  2015    11    3   Monday   
2015-11-30     CA_4  FOODS_3_555  d_1767  2015    11    3   Monday   
2015-11-30     CA_4  FOODS_3_586  d_1767  2015    11    3   Monday   
2015-11-30     CA_4  FOODS_3_587  d_1767  2015    11    3   Monday   
2015-11-30     CA_4  FOODS_3_714  d_1767  2015    11    3   Monday   

           event_name_1 event_type_1  wm_yr_wk  ventas  sell_price  
date                                                                
2013-01-01      NewYear     National     11249       0        1.25  
2013-01-01      NewYear     National     11249      33        1.25  
2013-01-01      NewYear     National     11249       0        4.98  
2013-01-01      NewYear     National     11249       0        4.98  
2013-01-01      NewYear     National     11249      20        4.28  
...                 ...          ...       ...     ...         ...  
2015-11-30   Sin_evento   Sin_evento     11544       9        1.68  
2015-11-30   Sin_evento   Sin_evento     11544      26        2.48  
2015-11-30   Sin_evento   Sin_evento     11544      13        2.48  
2015-11-30   Sin_evento   Sin_evento     11544      11        1.58  
2015-11-30   Sin_evento   Sin_evento     11544       9        1.58  

[21280 rows x 12 columns]

### Variable demanda intermitente

Esta variable va a identificar cuantos días seguidos han transcurrido con ventas cero.

La definiremos como que si los últimos n días han tenido cero ventas entonces hay rotura de stock.

Podemos crear varias cambiando el n.

Nos servirá para modelizar.

In [53]:
def rotura_stock(ventas, n = 5):
    cero_ventas = pd.Series(np.where(ventas == 0,1,0))
    num_ceros = cero_ventas.rolling(n).sum()
    rotura_stock = np.where(num_ceros == n,1,0)
    return(rotura_stock)

In [54]:
df = df.sort_values(by = ['store_id','item_id','date'])

In [55]:
df['rotura_stock_3'] = df.groupby(['store_id','item_id']).ventas.transform(lambda x: rotura_stock(x, 3)).values

In [56]:
df['rotura_stock_7'] = df.groupby(['store_id','item_id']).ventas.transform(lambda x: rotura_stock(x,7)).values

In [57]:
df['rotura_stock_15'] = df.groupby(['store_id','item_id']).ventas.transform(lambda x: rotura_stock(x,15)).values

### Variables de lag

Vamos a crear lags sobre las siguientes variables:

* ventas: lags de 15 días
* sell_price: lags de 7 días
* rotura_stock: lag de un día

In [58]:
def crear_lags(df, variable, num_lags = 7):
    
    #Crea el objeto dataframe
    lags = pd.DataFrame()
    
    #Crea todos los lags
    for cada in range(1,num_lags+1):
        lags[variable + '_lag_'+ str(cada)] = df[variable].shift(cada)
    
    #Devuelve el dataframe de lags
    return(lags)

In [59]:
lags_sell_price_df = df.groupby(['store_id','item_id'])\
                    .apply(lambda x: crear_lags(df = x, variable = 'sell_price', num_lags= 7))

In [60]:
lags_rotura_stock_3_df = df.groupby(['store_id','item_id'])\
                    .apply(lambda x: crear_lags(df = x, variable = 'rotura_stock_3', num_lags= 1))

In [61]:
lags_rotura_stock_7_df = df.groupby(['store_id','item_id'])\
                    .apply(lambda x: crear_lags(df = x, variable = 'rotura_stock_7', num_lags= 1))

In [62]:
lags_rotura_stock_15_df = df.groupby(['store_id','item_id'])\
                    .apply(lambda x: crear_lags(df = x, variable = 'rotura_stock_15', num_lags= 1))

In [63]:
lags_ventas_df = df.groupby(['store_id','item_id'])\
                    .apply(lambda x: crear_lags(df = x, variable = 'ventas', num_lags= 15))

### Variables de ventanas móviles

Vamos a crear tres tipos de ventanas móviles sobre las ventas:

* mínimo móvil
* media móvil
* máximo móvil

Cada uno de ellos en el rango de 15 días.

In [64]:
def min_movil(df, variable, num_periodos = 7):

    minm = pd.DataFrame()
    
    for cada in range(2,num_periodos+1):
        minm[variable + '_minm_' + str(cada)] = df[variable].shift(1).rolling(cada).min()
    
    #Devuelve el dataframe de lags
    return(minm)

In [65]:
def media_movil(df, variable, num_periodos = 7):

    mm = pd.DataFrame()
    
    for cada in range(2,num_periodos+1):
        mm[variable + '_mm_' + str(cada)] = df[variable].shift(1).rolling(cada).mean()
    
    #Devuelve el dataframe de lags
    return(mm)

In [66]:
def max_movil(df, variable, num_periodos = 7):

    maxm = pd.DataFrame()
    
    for cada in range(2,num_periodos+1):
        maxm[variable + '_maxm_' + str(cada)] = df[variable].shift(1).rolling(cada).max()
    
    #Devuelve el dataframe de lags
    return(maxm)

In [67]:
min_movil_df = df.groupby(['store_id','item_id'])\
                    .apply(lambda x: min_movil(df = x, variable = 'ventas', num_periodos= 15))

In [68]:
media_movil_df = df.groupby(['store_id','item_id'])\
                    .apply(lambda x: media_movil(df = x, variable = 'ventas', num_periodos= 15))

In [69]:
max_movil_df = df.groupby(['store_id','item_id'])\
                    .apply(lambda x: max_movil(df = x, variable = 'ventas', num_periodos= 15))

## PREPARAR LOS DATASETS

### Unir todos los dataframes generados

In [70]:
df_unido = pd.concat([df,
                      lags_sell_price_df,
                      lags_rotura_stock_3_df,
                      lags_rotura_stock_7_df,
                      lags_rotura_stock_15_df,
                      lags_ventas_df,
                      min_movil_df,
                      media_movil_df,
                      max_movil_df], axis = 1)

df_unido

store_id      item_id       d  year month wday    weekday  \
date                                                                   
2013-01-01     CA_3  FOODS_3_090   d_704  2013     1    4    Tuesday   
2013-01-02     CA_3  FOODS_3_090   d_705  2013     1    5  Wednesday   
2013-01-03     CA_3  FOODS_3_090   d_706  2013     1    6   Thursday   
2013-01-04     CA_3  FOODS_3_090   d_707  2013     1    7     Friday   
2013-01-05     CA_3  FOODS_3_090   d_708  2013     1    1   Saturday   
...             ...          ...     ...   ...   ...  ...        ...   
2015-11-26     CA_4  FOODS_3_714  d_1763  2015    11    6   Thursday   
2015-11-27     CA_4  FOODS_3_714  d_1764  2015    11    7     Friday   
2015-11-28     CA_4  FOODS_3_714  d_1765  2015    11    1   Saturday   
2015-11-29     CA_4  FOODS_3_714  d_1766  2015    11    2     Sunday   
2015-11-30     CA_4  FOODS_3_714  d_1767  2015    11    3     Monday   

            event_name_1 event_type_1  wm_yr_wk  ...  ventas_maxm_6  \
date                                             ...                  
2013-01-01       NewYear     National     11249  ...            NaN   
2013-01-02    Sin_evento   Sin_evento     11249  ...            NaN   
2013-01-03    Sin_evento   Sin_evento     11249  ...            NaN   
2013-01-04    Sin_evento   Sin_evento     11249  ...            NaN   
2013-01-05    Sin_evento   Sin_evento     11250  ...            NaN   
...                  ...          ...       ...  ...            ...   
2015-11-26  Thanksgiving     National     11543  ...           15.0   
2015-11-27    Sin_evento   Sin_evento     11543  ...           22.0   
2015-11-28    Sin_evento   Sin_evento     11544  ...           22.0   
2015-11-29    Sin_evento   Sin_evento     11544  ...           22.0   
2015-11-30    Sin_evento   Sin_evento     11544  ...           22.0   

            ventas_maxm_7  ventas_maxm_8  ventas_maxm_9  ventas_maxm_10  \
date                                                                      
2013-01-01            NaN            NaN            NaN             NaN   
2013-01-02            NaN            NaN            NaN             NaN   
2013-01-03            NaN            NaN            NaN             NaN   
2013-01-04            NaN            NaN            NaN             NaN   
2013-01-05            NaN            NaN            NaN             NaN   
...                   ...            ...            ...             ...   
2015-11-26           15.0           15.0           15.0            15.0   
2015-11-27           22.0           22.0           22.0            22.0   
2015-11-28           22.0           22.0           22.0            22.0   
2015-11-29           22.0           22.0           22.0            22.0   
2015-11-30           22.0           22.0           22.0            22.0   

            ventas_maxm_11  ventas_maxm_12  ventas_maxm_13  ventas_maxm_14  \
date                                                                         
2013-01-01             NaN             NaN             NaN             NaN   
2013-01-02             NaN             NaN             NaN             NaN   
2013-01-03             NaN             NaN             NaN             NaN   
2013-01-04             NaN             NaN             NaN             NaN   
2013-01-05             NaN             NaN             NaN             NaN   
...                    ...             ...             ...             ...   
2015-11-26            17.0            17.0            17.0            17.0   
2015-11-27            22.0            22.0            22.0            22.0   
2015-11-28            22.0            22.0            22.0            22.0   
2015-11-29            22.0            22.0            22.0            22.0   
2015-11-30            22.0            22.0            22.0            22.0   

            ventas_maxm_15  
date                        
2013-01-01             NaN  
2013-01-02             NaN  
2013-01-03             NaN  
2013-01-04    

### Eliminar los nulos que han generado las nuevas variables

In [71]:
df_unido.dropna(inplace=True)

### Eliminar las variables que no vamos a necesitar para modelizar

In [72]:
a_eliminar = ['d','wm_yr_wk','sell_price','rotura_stock_3','rotura_stock_7','rotura_stock_15']

In [73]:
df_unido.drop(columns=a_eliminar, inplace=True)

### Identificar la target

In [74]:
target = df_unido.ventas

### Separar num y cat

In [75]:
cat = df_unido.select_dtypes(include='O')

In [76]:
num = df_unido.select_dtypes(exclude='O')

## TRANSFORMACIÓN DE CATEGÓRICAS

### One Hot Encoding

#### Variables a aplicar OHE

In [77]:
var_ohe = ['year',
          'month',
          'wday',
          'weekday',
          'event_name_1',
          'event_type_1'
        ]

#### Instanciar

In [78]:
ohe = OneHotEncoder(sparse = False, handle_unknown='ignore')

#### Entrenar y aplicar

In [79]:
cat_ohe = ohe.fit_transform(cat[var_ohe])

#### Guardar como dataframe

In [80]:
cat_ohe = pd.DataFrame(cat_ohe, columns = ohe.get_feature_names_out())

### Target Encoding

#### Variables a aplicar TE

In [81]:
var_te = ['year',
          'month',
          'wday',
          'weekday',
          'event_name_1',
          'event_type_1'
        ]

#### Instanciar

In [82]:
te = TargetEncoder(min_samples_leaf=100, return_df = False)

#### Entrenar y aplicar

In [83]:
cat_te = te.fit_transform(cat[var_te], y = target)

#### Guardar como dataframe

In [84]:
#Añadir sufijos a los nombres
nombres_te = [variable + '_te' for variable in var_te]

#Guardar como dataframe
cat_te = pd.DataFrame(cat_te, columns = nombres_te)

## UNIFICAR DATASETS TRANSFORMADOS

### Meter en una lista todos los dataframes generados

Rescatamos de df_unido las variables de segmentación.

In [85]:
de_df_unido = df_unido[['store_id','item_id']].reset_index()

de_df_unido.head(2)

date store_id      item_id
0 2013-01-16     CA_3  FOODS_3_090
1 2013-01-17     CA_3  FOODS_3_090

### Unir todos los dataframes

In [86]:
dataframes = [de_df_unido, cat_ohe,cat_te,num.reset_index(drop=True)]

In [87]:
df_tablon = pd.concat(dataframes, axis = 1)

df_tablon

date store_id      item_id  year_2013  year_2014  year_2015  \
0     2013-01-16     CA_3  FOODS_3_090        1.0        0.0        0.0   
1     2013-01-17     CA_3  FOODS_3_090        1.0        0.0        0.0   
2     2013-01-18     CA_3  FOODS_3_090        1.0        0.0        0.0   
3     2013-01-19     CA_3  FOODS_3_090        1.0        0.0        0.0   
4     2013-01-20     CA_3  FOODS_3_090        1.0        0.0        0.0   
...          ...      ...          ...        ...        ...        ...   
20975 2015-11-26     CA_4  FOODS_3_714        0.0        0.0        1.0   
20976 2015-11-27     CA_4  FOODS_3_714        0.0        0.0        1.0   
20977 2015-11-28     CA_4  FOODS_3_714        0.0        0.0        1.0   
20978 2015-11-29     CA_4  FOODS_3_714        0.0        0.0        1.0   
20979 2015-11-30     CA_4  FOODS_3_714        0.0        0.0        1.0   

       month_1  month_2  month_3  month_4  ...  ventas_maxm_6  ventas_maxm_7  \
0          1.0      0.0      0.0      0.0  ...          351.0          351.0   
1          1.0      0.0      0.0      0.0  ...          351.0          351.0   
2          1.0      0.0      0.0      0.0  ...          351.0          351.0   
3          1.0      0.0      0.0      0.0  ...          281.0          351.0   
4          1.0      0.0      0.0      0.0  ...          362.0          362.0   
...        ...      ...      ...      ...  ...            ...            ...   
20975      0.0      0.0      0.0      0.0  ...           15.0           15.0   
20976      0.0      0.0      0.0      0.0  ...           22.0           22.0   
20977      0.0      0.0      0.0      0.0  ...           22.0           22.0   
20978      0.0      0.0      0.0      0.0  ...           22.0           22.0   
20979      0.0      0.0      0.0      0.0  ...           22.0           22.0   

       ventas_maxm_8  ventas_maxm_9  ventas_maxm_10  ventas_maxm_11  \
0              351.0          351.0           351.0           351.0   
1              351.0          351.0           351.0           351.0   
2              351.0          351.0           351.0           351.0   
3              351.0          351.0           351.0           351.0   
4              362.0          362.0           362.0           362.0   
...              ...            ...             ...             ...   
20975           15.0           15.0            15.0            17.0   
20976           22.0           22.0            22.0            22.0   
20977           22.0           22.0            22.0            22.0   
20978           22.0           22.0            22.0            22.0   
20979           22.0           22.0            22.0            22.0   

       ventas_maxm_12  ventas_maxm_13  ventas_maxm_14  ventas_maxm_15  
0               351.0           351.0           351.0           351.0  
1               351.0           351.0           351.0           351.0  
2               351.0           351.0           351.0           351.0  
3               351.0           351.0           351.0           351.0  
4               362.0           362.0           362.0           362.0  
...               ...             ...             ...             ...  
20975            17.0            17.0            17.0            17.0  
20976            22.0            22.0            22.0            22.0  
20977            22.0            22.0            22.0            22.0  
20978            22.0            22.0            22.0            22.0  
20979            22.0            22.0            22.0            22.0  

[20980 rows x 142 columns]

## GUARDAR DATASET TRAS TRANSFORMACIÓN DE DATOS

En formato pickle para no perder las modificaciones de metadatos.

In [88]:
#Definir los nombres del archivo
ruta_df_tablon = ruta_proyecto + '/02_Datos/03_Trabajo/' + 'df_tablon.pickle'

In [89]:
#Guardar los archivos
df_tablon.to_pickle(ruta_df_tablon)